<a href="https://colab.research.google.com/github/HugoAGFM/spark-data-analysis/blob/main/Data_Analysis_with_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brazil Highway Traffic Accidents Data Analysis Using Spark

## Introduction

This work proposes to briefly analyze a dataset containing Brazil highway traffic accidents data using Apache Spark. This is a continuation of a [data analysis using Apache Hive](https://github.com/HugoAGFM/hive_data_analysis), mainly focused on comparing both tools.

## About Dataset

The dataset under analysis is available in [kaggle](https://www.kaggle.com/datasets/mcamera/brazil-highway-traffic-accidents) and contains data from Brazil's federal higways between 2007 and 2021 at various levels of granularity.

The level of granularity chosen was 'accidents per person', and, because there's been  a change in the system used to record these data in 2017, this analysis will be restricted to the period of 2017 to 2021. The files are available [here](https://drive.google.com/drive/u/0/folders/1ue0XT3qt7dknBmV_oykird_6O2Rld723)


### Columns description

Below you can find the columns present in the dataset with a brief description, and how they've been mapped to each table in the data model.

| Coloumn                | Description                                                 | Table             |
| :--------------------- | :---------------------------------------------------------- | :---------------- |
| id                     | ID of accident                                              | Accidents         |
| data_inversa           | Date when accident hapened                                  | Accidents         |
| dia_semana             | Weekday when accident happened                              | Accidents         |
| horario                | Time when accident happened                                 | Accidents         |
| uf                     | Federal Unit where accident happened                        | Accidents         |
| br                     | Highway where accident happened                             | Accidents         |
| km                     | Highway kilometer where accident happened                  | Accidents         |
| municipio              | City where accident happened                                | Accidents         |
| causa_acidente         | Accident cause                                              | Accidents         |
| tipo_acidente          | Accident type                                               | Accidents         |
| classificacao_acidente | Accident classification                                     | Accidents         |
| fase_dia               | Pereiod of the day when  accident happened                  | Accidents         |
| sentido_via            | Lane direction where accident happened                      | Accidents         |
| condicao_metereologica | Weather condition when accident happened                    | Accidents         |
| tipo_pista             | Lane type where accident happened                           | Accidents         |
| tracado_via            | Lane layout where accident happened                         | Accidents         |
| uso_solo               | If the vehicle left the lane in the accident                                   | Accidents         |
| id_veiculo             | ID of vehicle involved in the accident                      | Accidents/Vehicle |
| tipo_veiculo           | Type of the vehicle involved in the accident                | Vehicle           |
| marca                  | Brand of the vehicle involved in the accident               | Vehicle           |
| ano_fabricacao_veiculo | Year of manufacture of the vehicle involved in the accident | Vehicle           |
| pesid                  | ID of person involved in the accident                       | Accidents/People  |
| tipo_envolvido         | Type of person involved in the accident                     | People            |
| estado_fisico          | Physical state of the person involved in the accident       | People            |
| idade                  | Age of the person involved in the accident                  | People            |
| sexo                   | Sex of the person involved in the accident                  | People            |

### Data model

For the purpose of this work and following the instructions of the professor, the dataset was denormalized and modeled as three different tables as described below.

| Table     | Description                                                                                                             |
| :-------- | :---------------------------------------------------------------------------------------------------------------------- |
| Accidents | Accidents records. Relates to table People through column 'pesid', and with table Vehicles through column 'id_veiculo'. |
| People    | Information about the people involved in the accidents. Relates to the table Accidents through column 'pesid'.           |
| Vehicles  | Information about the veicles involved in the accidents. Relates to the table Accidents thorugh column 'id_veiculo'.                                                |


## Preparing Environment

This session contains all the steps taken to prepare the analytical environment.

### Install pyspark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=578d653bbdc3f0580c106762a4cc16624698f72eea5b388bab4e5c9017720b75
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Mount Google Drive into Google Colab machine

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import and instanciate Spark Session

Spark Session consists of the entrypoint for the Spark SQL API. This work will run in a local environment.

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .getOrCreate()

## Data Analysis
Creating a dataframe for each table described in the Data Model

In [5]:
df_people = spark.read \
  .options(delimiter=";") \
  .csv("/content/drive/MyDrive/traffic/people.csv",
       inferSchema=True,
       header=True,
       encoding="Latin1")

df_accidents = spark.read \
  .options(delimiter=";") \
  .csv("/content/drive/MyDrive/traffic/accidents.csv",
       inferSchema=True,
       header=True,
       encoding="Latin1")

df_vehicles = spark.read \
  .options(delimiter=";") \
  .csv("/content/drive/MyDrive/traffic/vehicles.csv",
       inferSchema=True,
       header=True,
       encoding="Latin1")

In [6]:
from pyspark.sql.functions import when, col, count, sum, year, max

### Does the number of people in the vehicle influence the occurrences of accidents where the driver sleeps?

When compared to the total number of accidents per number of people in the vehicle, the ones where the driver sleeps are twice more common when there is only one person in the vehicle.

In [7]:
support_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .where("tipo_envolvido in ('Passageiro', 'Condutor')") \
                         .groupBy("id", "causa_acidente") \
                         .count() \
                         .withColumn("num_people", when(col("count") > 1, "Mais de uma"). \
                                     otherwise("Apenas uma"))

In [8]:
support_df.show(10)

+----+--------------------+-----+-----------+
|  id|      causa_acidente|count| num_people|
+----+--------------------+-----+-----------+
| 133|  Ingestão de Álcool|    2|Mais de uma|
| 465|  Ingestão de Álcool|    6|Mais de uma|
| 681|Desobediência às ...|    2|Mais de uma|
| 698|Avarias e/ou desg...|    1| Apenas uma|
|1326|Carga excessiva e...|    2|Mais de uma|
|1849|  Ingestão de Álcool|    4|Mais de uma|
|2745|Falta de Atenção ...|    1| Apenas uma|
|2870|   Condutor Dormindo|    4|Mais de uma|
|3158|Desobediência às ...|    2|Mais de uma|
|3171|Velocidade Incomp...|    1| Apenas uma|
+----+--------------------+-----+-----------+
only showing top 10 rows



In [9]:
final_df = support_df.withColumn("is_driver_sleeping",
                                 when(col("causa_acidente") == "Condutor Dormindo", 1). \
                                 otherwise(0)) \
                     .groupBy("num_people") \
                     .agg(sum("is_driver_sleeping").alias("num_accidents_w_sleeping_drivers"),
                          count("id").alias("total_num_accidents")) \
                     .withColumn("rate", col("num_accidents_w_sleeping_drivers") / col("total_num_accidents") * 100)

In [10]:
final_df.show()

+-----------+--------------------------------+-------------------+-----------------+
| num_people|num_accidents_w_sleeping_drivers|total_num_accidents|             rate|
+-----------+--------------------------------+-------------------+-----------------+
|Mais de uma|                            6111|             215042|2.841770444843333|
| Apenas uma|                            5971|             105690|5.649541110795724|
+-----------+--------------------------------+-------------------+-----------------+



### What is the vehicle type with the highest driver death rate?

Although the Quadricycle has the highest death rate, due to the low number of occurencies, it is possible to consider the Bicycle as the vehicle with highest death rate.

In [11]:
support_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid, "left") \
                         .join(df_vehicles, df_accidents.id_veiculo == df_vehicles.id_veiculo, "left") \
                         .where("tipo_envolvido = 'Condutor'") \
                         .groupBy("tipo_veiculo", "estado_fisico") \
                         .count()

In [12]:
support_df.show(10)

+----------------+-------------+-----+
|    tipo_veiculo|estado_fisico|count|
+----------------+-------------+-----+
|          Outros|        Ileso|  528|
|    Micro-ônibus|Lesões Graves|   89|
| Caminhão-trator|Lesões Graves| 2298|
| Trator de rodas| Lesões Leves|   42|
|    Micro-ônibus|        Óbito|   26|
|        Caminhão| Lesões Leves| 7155|
|          Ônibus|        Óbito|   55|
|          Ônibus|Não Informado|  195|
|Carroça-charrete|Não Informado|   36|
|     Semireboque|Lesões Graves|    9|
+----------------+-------------+-----+
only showing top 10 rows



In [13]:
final_df = support_df.withColumn("num_deaths",
                                 when(col("estado_fisico") == "Óbito", col("count")). \
                                 otherwise(0)) \
                     .groupBy("tipo_veiculo") \
                     .agg(sum("num_deaths").alias("num_deaths"),
                          sum("count").alias("total_drivers")) \
                     .withColumn("death_rate", col("num_deaths") / col("total_drivers") * 100) \
                     .orderBy(col("death_rate").desc())

In [14]:
final_df.show()

+----------------+----------+-------------+-------------------+
|    tipo_veiculo|num_deaths|total_drivers|         death_rate|
+----------------+----------+-------------+-------------------+
|     Quadriciclo|         2|            9|  22.22222222222222|
|       Bicicleta|       932|         8787| 10.606577899169228|
|    Carro de mão|         2|           25|                8.0|
|      Ciclomotor|       117|         1661|  7.043949428055388|
|     Motocicleta|      5626|       107284|  5.244025204131091|
|Carroça-charrete|        12|          257|  4.669260700389105|
|        Motoneta|       391|        12580| 3.1081081081081083|
| Trator de rodas|         8|          288| 2.7777777777777777|
|        Caminhão|      1005|        44956|  2.235519174303764|
|       Automóvel|      4332|       216772| 1.9984130791799681|
|     Caminhonete|       933|        46863|  1.990909672876256|
| Caminhão-trator|      1014|        55104| 1.8401567944250872|
|        Triciclo|         1|           

### Which federal unit has the highest death rate by person involved in an accident?

Maranhão is the State with the highest death rate per people involved in an accident. Beyond that, it is important to notice that mostly States from the North and Northeast regions are in the top of the list.

In [15]:
final_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid, "left") \
                       .withColumn("is_dead",
                                 when(col("estado_fisico") == "Óbito", 1). \
                                 otherwise(0)) \
                       .groupBy("uf") \
                       .agg(sum("is_dead").alias("num_deaths"),
                            count("id").alias("num_people")) \
                       .withColumn("death_rate", col("num_deaths") / col("num_people") * 100) \
                       .orderBy(col("death_rate").desc())

In [16]:
final_df.show()

+---+----------+----------+------------------+
| uf|num_deaths|num_people|        death_rate|
+---+----------+----------+------------------+
| MA|      1022|     14081| 7.258007243803707|
| AL|       418|      7228| 5.783065855008301|
| TO|       380|      6718| 5.656445370646026|
| BA|      2284|     41653| 5.483398554725951|
| PA|       647|     12424| 5.207662588538313|
| PI|       728|     14081| 5.170087351750586|
| AM|        70|      1446| 4.840940525587829|
| PE|      1405|     29409|4.7774490802135405|
| RR|       133|      2913| 4.565739787161002|
| CE|       796|     17964| 4.431084391004231|
| MT|       972|     25853| 3.759718407921711|
| MS|       595|     17200|3.4593023255813953|
| GO|      1298|     37623| 3.450017276665869|
| PB|       557|     16383| 3.399865714460111|
| RN|       485|     14380|3.3727399165507648|
| SE|       211|      6432|3.2804726368159205|
| MG|      3247|    100693|3.2246531536452383|
| AC|        91|      2964| 3.070175438596491|
| PR|      24

### What are the stages of the day where an accident involving someone in an altered mental stage is more likely to happen?

When compared to the total number of accidents, the ones where someone is in an altered menthal state are more likely to happen during the night or dawn. It is important to notice that the occurences involve not only ingestion of alcohol, but other substances as well, and includes ingestion from drivers and pedestrians.

In [17]:
final_df = df_accidents.select("id", "fase_dia", "causa_acidente") \
                       .distinct() \
                       .withColumn("is_altered_mental_state",
                                   when(col("causa_acidente").like("Ingest%"), 1). \
                                   otherwise(0)) \
                       .groupBy("fase_dia") \
                       .agg(sum("is_altered_mental_state").alias("num_accidents_w_altered_mental_state"),
                            count("id").alias("total_num_accidents")) \
                       .withColumn("accidents_rate", col("num_accidents_w_altered_mental_state") / col("total_num_accidents") * 100) \
                       .orderBy(col("accidents_rate").desc())

In [18]:
final_df.show()

+-----------+------------------------------------+-------------------+------------------+
|   fase_dia|num_accidents_w_altered_mental_state|total_num_accidents|    accidents_rate|
+-----------+------------------------------------+-------------------+------------------+
|Plena Noite|                               16096|             111170| 14.47872627507421|
|  Amanhecer|                                1500|              15588|  9.62278675904542|
|  Anoitecer|                                1343|              17708| 7.584142760334313|
|  Pleno dia|                                6965|             176269|3.9513470888244675|
+-----------+------------------------------------+-------------------+------------------+



### What is the highway with the highest number of accidents each year?

The biggest highway in the Country, BR 101, is also the one with the highest number of accidents every year. It is important to notice that this number decreased throughout the years.

In [19]:
accidents_by_road_df = df_accidents.withColumn("year", year("data_inversa")) \
                                   .select("id", "year", "br", "id_veiculo") \
                                   .join(df_vehicles, df_accidents.id_veiculo == df_vehicles.id_veiculo, "left") \
                                   .where("tipo_veiculo = 'Automóvel'") \
                                   .groupBy("year", "br") \
                                   .count()

max_accidents_by_road_df = accidents_by_road_df.groupBy("year").agg(max("count").alias("max_num_accidents"))

In [20]:
accidents_by_road_df.show(10)

+----+---+-----+
|year| br|count|
+----+---+-----+
|2017|423|  219|
|2018|488|   41|
|2018|428|  103|
|2018|475|    3|
|2017|104|  598|
|2018| 70| 1095|
|2018|451|    1|
|2017| 20| 1359|
|2018|435|   78|
|2017|242|  452|
+----+---+-----+
only showing top 10 rows



In [21]:
max_accidents_by_road_df.show()

+----+-----------------+
|year|max_num_accidents|
+----+-----------------+
|2018|            11505|
|2019|            11348|
|2020|            10563|
|2017|            14832|
|2021|             5454|
+----+-----------------+



In [22]:
df_final = max_accidents_by_road_df.join(accidents_by_road_df,
                                         max_accidents_by_road_df["max_num_accidents"] == accidents_by_road_df["count"]) \
                                   .select(max_accidents_by_road_df["year"], "br", "max_num_accidents") \
                                   .orderBy(col("year"))

In [23]:
df_final.show()

+----+---+-----------------+
|year| br|max_num_accidents|
+----+---+-----------------+
|2017|101|            14832|
|2018|101|            11505|
|2019|101|            11348|
|2020|101|            10563|
|2021|101|             5454|
+----+---+-----------------+



## Extended Data Analysis

This section intends to understand how different attributes in the dataset are related with the death rate. Furtheremore, the conclusions of it will be used to determine which atributes can be used to create a classification model.

### People attributes

In [24]:
df_people.printSchema()

root
 |-- pesid: integer (nullable = true)
 |-- tipo_envolvido: string (nullable = true)
 |-- estado_fisico: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- sexo: string (nullable = true)



#### People type

Pedrestrians are more likely to die on traffic accidents.

In [25]:
people_type_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("tipo_envolvido") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100)

people_type_df.show()

+--------------+----------------------------+-----------------------+--------------------+
|tipo_envolvido|num_dead_people_in_accidents|num_people_in_accidents|          death_rate|
+--------------+----------------------------+-----------------------+--------------------+
| Não Informado|                           0|                      5|                 0.0|
|    Passageiro|                        5827|                 192952|   3.019922053153116|
|    Testemunha|                           1|                  15057|0.006641429235571...|
|      Pedestre|                        4071|                  16005|   25.43580131208997|
|      Condutor|                       14692|                 525851|  2.7939473348914428|
|     Cavaleiro|                          19|                    178|  10.674157303370785|
+--------------+----------------------------+-----------------------+--------------------+



#### Gender

Men dying in traffic accidents are 36% more common.

In [26]:
gender_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("sexo") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100)

gender_df.show(50)

+-------------+----------------------------+-----------------------+-------------------+
|         sexo|num_dead_people_in_accidents|num_people_in_accidents|         death_rate|
+-------------+----------------------------+-----------------------+-------------------+
|     Ignorado|                          63|                   4375|               1.44|
|Não Informado|                          75|                  45255|0.16572754391779915|
|     Feminino|                        4479|                 164319| 2.7257955562046994|
|    Masculino|                       19993|                 536099| 3.7293484971992115|
+-------------+----------------------------+-----------------------+-------------------+



### Vehicles attributes

In [27]:
df_vehicles.printSchema()

root
 |-- id_veiculo: integer (nullable = true)
 |-- tipo_veiculo: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- ano_fabricacao_veiculo: integer (nullable = true)



#### Vehicle type

There are some vehicles that are significantly more likely to heve dead people involved. For exdample, motorcycles are 77% more likely to have dead people involved in accidents, when compared to cars.

In [29]:
vehicle_type_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .join(df_vehicles, df_accidents.id_veiculo == df_vehicles.id_veiculo) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("tipo_veiculo") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100) \
                         .orderBy("death_rate", ascending=False)

vehicle_type_df.show()

+----------------+----------------------------+-----------------------+------------------+
|    tipo_veiculo|num_dead_people_in_accidents|num_people_in_accidents|        death_rate|
+----------------+----------------------------+-----------------------+------------------+
|     Quadriciclo|                           2|                      9| 22.22222222222222|
|          Outros|                         521|                   3085|16.888168557536467|
|       Bicicleta|                         943|                   9521| 9.904421804432308|
|    Carro de mão|                           3|                     32|             9.375|
|    Trator misto|                           1|                     15| 6.666666666666667|
|      Ciclomotor|                         134|                   2092| 6.405353728489484|
|     Semireboque|                          67|                   1204| 5.564784053156146|
|Carroça-charrete|                          20|                    388| 5.154639175257731|

### Accidents attributes

In [30]:
df_accidents.printSchema()

root
 |-- id: integer (nullable = true)
 |-- data_inversa: date (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: timestamp (nullable = true)
 |-- uf: string (nullable = true)
 |-- br: integer (nullable = true)
 |-- km: double (nullable = true)
 |-- municipio: string (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- tipo_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- fase_dia: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_pista: string (nullable = true)
 |-- tracado_via: string (nullable = true)
 |-- uso_solo: string (nullable = true)
 |-- pesid: integer (nullable = true)
 |-- id_veiculo: integer (nullable = true)



#### Federal Unit

Regarding the federal units death rate range is wide varying from 1.74% to 7.26%.

In [40]:
federal_unit_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("uf") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100) \
                         .orderBy("death_rate", ascending=False)

federal_unit_df.show(50)

+---+----------------------------+-----------------------+------------------+
| uf|num_dead_people_in_accidents|num_people_in_accidents|        death_rate|
+---+----------------------------+-----------------------+------------------+
| MA|                        1022|                  14081| 7.258007243803707|
| AL|                         418|                   7228| 5.783065855008301|
| TO|                         380|                   6718| 5.656445370646026|
| BA|                        2284|                  41653| 5.483398554725951|
| PA|                         647|                  12424| 5.207662588538313|
| PI|                         728|                  14081| 5.170087351750586|
| AM|                          70|                   1446| 4.840940525587829|
| PE|                        1405|                  29409|4.7774490802135405|
| RR|                         133|                   2913| 4.565739787161002|
| CE|                         796|                  17964| 4.431

#### Accident type

Hitting pedestrians and frontal colision have significantly higher death rates when copared to other accident types.

In [42]:
accident_type_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("tipo_acidente") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100) \
                         .orderBy("death_rate", ascending=False)

accident_type_df.show(50, truncate=False)

+-------------------------------+----------------------------+-----------------------+-------------------+
|tipo_acidente                  |num_dead_people_in_accidents|num_people_in_accidents|death_rate         |
+-------------------------------+----------------------------+-----------------------+-------------------+
|Atropelamento de Pedestre      |4140                        |34997                  |11.829585393033689 |
|Colisão frontal                |7504                        |69618                  |10.77882156913442  |
|Colisão com objeto             |154                         |4112                   |3.745136186770428  |
|Atropelamento de Animal        |361                         |10850                  |3.3271889400921655 |
|Colisão lateral sentido oposto |71                          |2177                   |3.261368856224162  |
|Eventos atípicos               |8                           |251                    |3.187250996015936  |
|Colisão com objeto estático    |1129

#### Phase day

Deaths are more common in accidents during dawn or night.

In [44]:
day_phase_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("fase_dia") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100) \
                         .orderBy("death_rate", ascending=False)

day_phase_df.show()

+-----------+----------------------------+-----------------------+-----------------+
|   fase_dia|num_dead_people_in_accidents|num_people_in_accidents|       death_rate|
+-----------+----------------------------+-----------------------+-----------------+
|  Amanhecer|                        1740|                  34836|4.994832931450223|
|Plena Noite|                       11940|                 253485|4.710337889815965|
|  Anoitecer|                        1256|                  43797|2.867776331712218|
|  Pleno dia|                        9674|                 417930|2.314741703156031|
+-----------+----------------------------+-----------------------+-----------------+



#### Wheather condition

Deaths are significantly more common on accidents with a foggy weather.

In [45]:
weather_df = df_accidents.join(df_people, df_accidents.pesid == df_people.pesid) \
                         .withColumn("is_dead",
                                     when(col("estado_fisico") == "Óbito", 1). \
                                     otherwise(0)) \
                         .groupBy("condicao_metereologica") \
                         .agg(sum("is_dead").alias("num_dead_people_in_accidents"),
                              count("id").alias("num_people_in_accidents")) \
                         .withColumn("death_rate",
                                      col("num_dead_people_in_accidents") / col("num_people_in_accidents") * 100) \
                         .orderBy("death_rate", ascending=False)

weather_df.show()

+----------------------+----------------------------+-----------------------+------------------+
|condicao_metereologica|num_dead_people_in_accidents|num_people_in_accidents|        death_rate|
+----------------------+----------------------------+-----------------------+------------------+
|               Granizo|                           1|                     16|              6.25|
|      Nevoeiro/Neblina|                         362|                   6671| 5.426472792684755|
|              Ignorado|                         496|                   9971| 4.974425834921272|
|                 Vento|                          59|                   1490|3.9597315436241614|
|             Céu Claro|                       14764|                 428078|3.4489041716696494|
|               Nublado|                        4162|                 130085|3.1994465157397087|
|                 Chuva|                        2693|                  89703| 3.002129248743074|
|        Garoa/Chuvisco|      